In [6]:
%%bq query
#standardsql
-- Find the weather stations located in Finland and in daterange from bike data
create or replace table
`bergqvist-sandbox.bike_demo.gsodrel`
AS
with daterange as ( -- get first and last dates in bike data, use to filter weather data
select
  DATE(min(`timestamp`)) as start_bike,
  DATE(max(`timestamp`)) as end_bike
from `bergqvist-sandbox.bike_demo.hist`
)
SELECT 
   gsod.* except (stn),
   fwban.* except (usaf),
   ST_GeogPoint(fwban.lon, fwban.lat) AS geo
FROM 
(
SELECT
   usaf,
   lon,
   lat,
   name
FROM `bergqvist-sandbox.bike_demo.gsodstations` where country = "FI"
) fwban
join
(
SELECT
    stn, 
    DATE(cast(year as int64),
         cast(mo as int64),
         cast(da as int64)
         ) as `date`,
    temp as temp_f,
    wdsp as wind_speed,
    mxpsd as max_speed,
    gust as wind_gust,
    prcp as prepcip,
    flag_prcp as prep_type,
    sndp as snow_dep
FROM (
  select * from `bergqvist-sandbox.bike_demo.gsod2017` 
  union all 
  select * from `bergqvist-sandbox.bike_demo.gsod2018`
) hist_weather
) gsod
ON fwban.usaf = gsod.stn
where `date` between (select start_bike from daterange) and (select end_bike from daterange)
;


date,temp_f,wind_speed,max_speed,wind_gust,prepcip,prep_type,snow_dep,lon,lat,name,geo
2018-01-17,14.5,10.4,15.9,999.9,99.99,,999.9,25.355,64.93,OULU,POINT(25.355 64.93)
2018-01-19,10.9,5.5,8.9,999.9,99.99,,999.9,25.355,64.93,OULU,POINT(25.355 64.93)
2018-01-21,8.4,3.5,6.0,999.9,99.99,,999.9,25.355,64.93,OULU,POINT(25.355 64.93)
2018-03-24,17.1,6.8,12.0,999.9,99.99,,999.9,25.355,64.93,OULU,POINT(25.355 64.93)
2018-01-11,14.3,5.6,8.9,999.9,99.99,,999.9,25.355,64.93,OULU,POINT(25.355 64.93)
2017-06-12,56.3,6.4,9.9,999.9,99.99,,999.9,25.355,64.93,OULU,POINT(25.355 64.93)
2017-06-21,48.1,9.9,15.9,999.9,99.99,,999.9,25.355,64.93,OULU,POINT(25.355 64.93)
2017-12-04,23.4,5.4,9.9,999.9,99.99,,999.9,25.355,64.93,OULU,POINT(25.355 64.93)
2018-06-29,55.2,11.5,15.0,21.0,99.99,,999.9,25.355,64.93,OULU,POINT(25.355 64.93)
2018-01-05,28.9,8.0,9.9,999.9,99.99,,999.9,25.355,64.93,OULU,POINT(25.355 64.93)


In [12]:
%%bq query
#standardsql
-- Get unique list of station IDs with latest name and latest loction
CREATE OR REPLACE TABLE `bdh-test.helsinki_bikes.bike_stands` AS
SELECT
    * except (row_num),
    ST_GeogPoint(lon, lat)
AS
    geo
from (
SELECT
    name,
    stationid,
    ROW_NUMBER() OVER (PARTITION BY stationid ORDER BY `timestamp` DESC) AS row_num,
    lon,
    lat
FROM `bergqvist-sandbox.bike_demo.hist`
) tmp1
WHERE row_num = 1 and stationid != 0
;


accessDenied: Access Denied: Dataset bdh-test:helsinki_bikes: The user 168869582278-compute@developer.gserviceaccount.com does not have bigquery.tables.create permission for dataset bdh-test:helsinki_bikes.

In [8]:
%%bq query
#standardsql
-- Join bike_stands with gsodrel to find closet weatherstation to each bike stand
CREATE OR REPLACE TABLE `bergqvist-sandbox.bike_demo.stand_gsod_pairing` AS
WITH pre_filter_distance AS (
SELECT bike_stands.*,
  gsod.name AS gsodname,
  ST_Distance(bike_stands.geo, gsod.geo)/1000 distkm
FROM
  `bergqvist-sandbox.bike_demo.bike_stands` bike_stands
CROSS JOIN
  (SELECT name, any_value(geo) AS geo FROM `bergqvist-sandbox.bike_demo.gsodrel` GROUP BY name) gsod
WHERE ST_DWithin(bike_stands.geo, gsod.geo, 8000)
) 
SELECT 
 name, stationid, gsodname, distkm
FROM (
SELECT *, ROW_NUMBER() OVER (PARTITION BY stationid ORDER BY distkm ASC) AS row_num
FROM
pre_filter_distance ) tab
WHERE row_num = 1

name,stationid,gsodname,distkm
Melkonkuja,50,HELSINKI KAISANIEMI,5.10971288754
Jäähalli,80,HELSINKI KAISANIEMI,1.77454937188
Gebhardinaukio,119,HELSINKI KAISANIEMI,0.93769821765
Friisinkalliontie,152,ESPOO TAPIOLA,3.57128843058
Sepetlahdentie,609,ESPOO TAPIOLA,4.11803035437
Lauttasaaren ostoskeskus,57,HELSINKI KAISANIEMI,4.54856701513
Eteläinen Hesperiankatu,72,HELSINKI KAISANIEMI,2.04967023926
Seurasaari,91,HELSINKI KAISANIEMI,3.65772237928
Arielinkatu,125,HELSINKI KAISANIEMI,1.59595875769
Sateentie,511,ESPOO TAPIOLA,1.86494701294


In [9]:
%%bq query
#standardsql
-- combine bike with weather data at each location
CREATE OR REPLACE TABLE `bergqvist-sandbox.bike_demo.bike_final`
AS
SELECT
bikes.*,
   ST_GeogPoint(bikes.lon, bikes.lat) AS bikegeo,
   meteo.* EXCEPT (`date`, name, stationId)
FROM 
(
SELECT
   gsod.date, gsod.temp_f, gsod.wind_speed, gsod.max_speed, gsod.wind_gust,
   gsod.prepcip, gsod.prep_type, gsod.snow_dep, gsod.lon AS gsodlon, gsod.lat AS gsodlt, gsod.geo AS gsodgeo,
   local.*,
   RAND() as `rand` -- for modelling step
FROM `bergqvist-sandbox.bike_demo.gsodrel` gsod
JOIN
   `bergqvist-sandbox.bike_demo.stand_gsod_pairing` local
ON local.gsodname = gsod.name
) meteo
JOIN
   (
   SELECT
   *,
   EXTRACT(date FROM TIMESTAMP_SECONDS(UNIX_SECONDS(`timestamp`)) AT time zone "Europe/Helsinki") AS `date`,
   EXTRACT(dayofweek FROM TIMESTAMP_SECONDS(UNIX_SECONDS(`timestamp`)) AT time zone "Europe/Helsinki") AS dow,
   EXTRACT(month FROM TIMESTAMP_SECONDS(UNIX_SECONDS(`timestamp`)) AT time zone "Europe/Helsinki") AS month,
   EXTRACT(quarter FROM TIMESTAMP_SECONDS(UNIX_SECONDS(`timestamp`)) AT time zone "Europe/Helsinki") AS quarter,
   EXTRACT(hour FROM TIMESTAMP_SECONDS(UNIX_SECONDS(`timestamp`)) AT time zone "Europe/Helsinki") AS hour
   FROM
   `bergqvist-sandbox.bike_demo.hist`
   ) bikes
ON bikes.date = meteo.date AND bikes.stationId = meteo.stationId
;

stationId,name,timestamp,lon,lat,spacesAvailable,bikesAvailable,date,dow,month,quarter,hour,bikegeo,temp_f,wind_speed,max_speed,wind_gust,prepcip,prep_type,snow_dep,gsodlon,gsodlt,gsodgeo,gsodname,distkm,rand
76,Olympiastadion,2017-08-28 05:56:01,24.9249741,60.1845055,18,3,2017-08-28,2,8,3,8,POINT(24.9249741 60.1845055),54.1,3.9,7.2,12.8,0.0,I,999.9,24.95,60.183,POINT(24.95 60.183),HELSINKI KAISANIEMI,1.39373286282,0.212447999985
45,Brahen kenttä,2017-05-15 03:51:01,24.950833,60.186742,15,5,2017-05-15,2,5,2,6,POINT(24.950833 60.186742),47.8,7.3,11.5,20.8,99.99,,999.9,24.95,60.183,POINT(24.95 60.183),HELSINKI KAISANIEMI,0.418632956435,0.98178594719
50,Melkonkuja,2017-05-25 07:33:01,24.8861094,60.1498127,16,4,2017-05-25,5,5,2,10,POINT(24.8861094 60.1498127),56.4,6.9,13.6,25.1,99.99,,999.9,24.95,60.183,POINT(24.95 60.183),HELSINKI KAISANIEMI,5.10971288754,0.0677857116209
739,Postipuun koulu,2018-07-09 21:27:01,24.826396,60.223827,1,0,2018-07-10,3,7,3,0,POINT(24.826396 60.223827),61.3,4.1,7.8,999.9,99.99,,999.9,24.783,60.183,POINT(24.783 60.183),ESPOO TAPIOLA,5.13411843566,0.314794349236
525,Mäntyviita,2018-07-08 22:54:01,24.810974,60.178702,0,13,2018-07-09,2,7,3,1,POINT(24.810974 60.178702),63.4,4.7,9.7,999.9,0.0,E,999.9,24.783,60.183,POINT(24.783 60.183),ESPOO TAPIOLA,1.61892486384,0.291706170041
64,Tyynenmerenkatu,2018-06-21 21:48:01,24.9217464,60.1565255,23,5,2018-06-22,6,6,2,0,POINT(24.9217464 60.1565255),53.5,10.0,17.5,999.9,0.94,F,999.9,24.95,60.183,POINT(24.95 60.183),HELSINKI KAISANIEMI,3.33292506743,0.326162755365
142,Koskelantie,2017-09-03 02:04:01,24.9465139,60.208749,12,2,2017-09-03,1,9,3,5,POINT(24.9465139 60.208749),54.9,11.1,15.5,26.0,99.99,,999.9,24.967,60.2,POINT(24.967 60.2),HELSINKI KUMPULA,1.49254856822,0.669706777848
111,Esterinportti,2017-10-08 19:05:01,24.9267808,60.1975724,13,7,2017-10-08,1,10,4,22,POINT(24.9267808 60.1975724),49.3,10.2,15.3,26.8,0.59,F,999.9,24.95,60.183,POINT(24.95 60.183),HELSINKI KAISANIEMI,2.06712259578,0.780455419083
148,Juhana Herttuan tie,2017-10-28 16:14:01,24.9664619,60.2184427,6,11,2017-10-28,7,10,4,19,POINT(24.9664619 60.2184427),38.4,11.2,17.7,28.4,99.99,,999.9,24.967,60.2,POINT(24.967 60.2),HELSINKI KUMPULA,2.0509533479,0.25737256494
144,Käpyläntie,2018-05-19 03:36:01,24.9588108,60.2139043,0,19,2018-05-19,7,5,2,6,POINT(24.9588108 60.2139043),58.0,6.5,10.7,17.3,0.0,I,999.9,24.967,60.2,POINT(24.967 60.2),HELSINKI KUMPULA,1.61093254802,0.655393830369


In [10]:
%%bq query
#standardsql
-- Execute in https://bigquerygeoviz.appspot.com/

-- Plot locations for weather stations and bike stations
SELECT
  ANY_VALUE(bikegeo) AS loc,
  name,
  'Bike Station' AS type
FROM
  `bergqvist-sandbox.bike_demo.bike_final`
GROUP BY
  name
UNION ALL
SELECT
  ANY_VALUE(gsodgeo) AS loc,
  gsodname AS name,
  'Weather Station' AS type
FROM
  `bergqvist-sandbox.bike_demo.bike_final`
GROUP BY
  gsodname
;

loc,name,type
POINT(24.8759851 60.1983016),Laajalahden aukio,Bike Station
POINT(24.8713963 60.1948358),Torpanranta,Bike Station
POINT(24.924867 60.198898),Maistraatintori,Bike Station
POINT(24.8708833 60.2038803),Ulvilantie,Bike Station
POINT(24.767831 60.162477),Hauenkallio,Bike Station
POINT(24.796952 60.187281),Louhentori,Bike Station
POINT(24.9054609 60.2024393),Tenholantie,Bike Station
POINT(24.9554618 60.179765),Näkinsilta,Bike Station
POINT(24.824954 60.210735),Majurinkulma,Bike Station
POINT(24.9265905 60.16957343),Leppäsuonaukio,Bike Station


In [11]:
%%bq query
#standardsql
-- Execute in https://bigquerygeoviz.appspot.com/

-- Plot bike stations coloured by nearest weather station
SELECT
  ANY_VALUE(bikegeo) AS loc,
  name,
  ANY_VALUE(gsodname) AS gsod,
  avg(spacesAvailable + bikesAvailable) AS `size`
FROM
  `bergqvist-sandbox.bike_demo.bike_final`
GROUP BY
  name
;

loc,name,gsod,size
POINT(24.826396 60.223827),Postipuun koulu,ESPOO TAPIOLA,5.43260849407
POINT(24.810974 60.178702),Mäntyviita,ESPOO TAPIOLA,10.2322047156
POINT(24.750928 60.158083),Matinkartanontie,ESPOO TAPIOLA,25.5664434774
POINT(24.803942 60.177588),Kulttuuriaukio,ESPOO TAPIOLA,30.047181894
POINT(24.791754 60.158194),Westendintie,ESPOO TAPIOLA,16.3543412672
POINT(24.826671 60.184312),"Aalto-yliopisto (M), Korkeakouluaukio",ESPOO TAPIOLA,40.8789551046
POINT(24.79139 60.173567),Hakalehto,ESPOO TAPIOLA,23.9153751601
POINT(24.782536 60.159601),Toppelundintie,ESPOO TAPIOLA,12.8780525542
POINT(24.78576 60.182976),Koivu-Mankkaa,ESPOO TAPIOLA,20.4074971849
POINT(24.805613 60.214193),Säterinrinne,ESPOO TAPIOLA,7.76231427181
